In [1]:
import csv

f = open("H&M_boys.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","composition","color","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()


In [2]:
!wget -O "auth.json" https://dl.dropbox.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json

--2019-07-19 20:18:38--  https://dl.dropbox.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.1.6, 2620:100:6016:6::a27d:106
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://dl.dropboxusercontent.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json [following]
--2019-07-19 20:18:38--  https://dl.dropboxusercontent.com/s/p1ifzzf2443r05g/urbankiddo-c8528ce11ed0.json
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.1.6, 2620:100:6016:6::a27d:106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2315 (2.3K) [text/plain]
Saving to: ‘auth.json’

auth.json           100%[===================>]   2.26K  --.-KB/s    in 0s      

2019-07-19 20:18:39 (43.6 MB/s) - ‘auth.json’ saved [2315/2315]



In [3]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [4]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests

ur="https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


r=requests.get(ur, headers=headers)
html_doc=r.content.decode()
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

In [6]:
filters=soup.find_all("div", attrs={ "id" :"drawer-productTypes"})
soup2 = BeautifulSoup(str(filters), 'html.parser')
#print(soup.prettify())
pr=[]
cnt=[]
prods=soup2.find_all("li", attrs={ "class" :"inputwrapper level-list-item"}) #loop
for subc in prods:
    pr.append(subc.find("span",attrs={ "class" :"text"}).text)
    cnt.append(subc.find("span",attrs={ "class" :"item-count"}).text)

In [7]:
list(zip(pr,cnt))

[('badbyxor', '3'),
 ('byxor', '75'),
 ('cardigan', '1'),
 ('diadem', '1'),
 ('dräkt', '4'),
 ('halsduk', '15'),
 ('handskar', '1'),
 ('hatt', '2'),
 ('hängslen', '1'),
 ('jacka', '29'),
 ('jeans', '57'),
 ('kalsonger', '28'),
 ('kavaj', '9'),
 ('keps', '23'),
 ('kostymväst', '6'),
 ('leksak', '1'),
 ('maskeradaccessoar', '1'),
 ('morgonrock', '1'),
 ('mössa', '29'),
 ('pyjamas', '29'),
 ('shorts', '28'),
 ('skjorta', '39'),
 ('skor', '34'),
 ('skärp', '1'),
 ('stickad tröja', '14'),
 ('strumpor', '33'),
 ('t-shirt', '107'),
 ('topp', '17'),
 ('tröja', '81'),
 ('vantar', '1'),
 ('väska', '4'),
 ('väst', '4')]

In [8]:
!mkdir 'H&M' && ls

mkdir: cannot create directory ‘H&M’: File exists


In [9]:
ur

'https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html'

In [ ]:
import urllib.request as req
import time
import random

base=str("https://www2.hm.com")
brand='H&M'
gender='boy'
size='134-170'

for p,c in zip(pr,cnt):
    try:
        category=p
        p=p.replace(' ','+')
        url2=ur+str('?productTypes=')+p+str('&page-size=')+c
        print(url2)
        s=requests.get(url2,headers=headers)
        html_doc2=s.content.decode()
        soup2 = BeautifulSoup(html_doc2, 'html.parser')
        products=soup2.find_all("div", attrs={ "class" :"image-container"})
        print(len(products))

        for product in products:
            ext=product.a['href']
            name=product.a["title"]
            url3=base+ext
            p=requests.get(url3,headers=headers)
            html_doc3=p.content.decode()
            soup3 = BeautifulSoup(html_doc3, 'html.parser')
            sub_product=soup3.find("div", attrs={ "class" :"mini-slider"})
            html_doc4=str(sub_product)
            soup4=BeautifulSoup(html_doc4, 'html.parser')
            spa=soup4.find_all("li", attrs={ "class" :"list-item"})
            for sp in spa:        
                ext2=sp.a['href']
                color=sp.a['title']
                url5=base+ext2
                spr=requests.get(url5,headers=headers)
                html_doc5=spr.content.decode()
                soup5 = BeautifulSoup(html_doc5, 'html.parser')

                iu=soup5.find("figure", attrs={ "class" :"pdp-image product-detail-images product-detail-main-image"})
                img_url=iu.div.img['src']
                desc=soup5.find("section", attrs={ "class" :"name-price"})
                price=desc.span.text.strip()
                qy=soup5.find("div", attrs={ "class" :"js-before-secondary-images"})
                composition=qy.div.ul.li.ul.li.text      
                a=ext2
                article_no=a.lstrip('/sv_se/productpage.').rstrip('.html')
                url=url5

                img_url = "https:" + img_url
                img_url=img_url.replace(' ','+')
                ac='H&M'+'/'+article_no+str('.jpg')
                req.urlretrieve(img_url,ac)

                blob_name=str('H&M')+str('/')+str(article_no)
                path_to_file='H&M/'+article_no+str('.jpg')
                blob_url=upload_to_bucket(blob_name, path_to_file)
                with open("H&M_boys.csv", "a", encoding='utf-8') as f:
                          writer = csv.writer(f)
                          writer.writerow([name,brand,gender,category,size,price,composition,color,url,img_url,article_no,blob_url])
    except:
        continue
    


https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=badbyxor&page-size=3
3
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=byxor&page-size=75
75
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=cardigan&page-size=1
1
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=diadem&page-size=1
1
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=dräkt&page-size=4
4
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=halsduk&page-size=15
15
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=handskar&page-size=1
1
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=hatt&page-size=2
2
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?productTypes=hängslen&page-size=1
1
https://www2.hm.com/sv_se/barn/produkter/pojke-8-14-ar-134-170.html?produ

In [14]:
!ls

auth.json	    H&M-baby-girl.ipynb  H&M.csv	  H&M.ipynb
H&M		    H&M_boy.csv		 H&M_girl.csv	  tutorials
H&M_baby_boy.csv    H&M_boy.ipynb	 H&M_girl.ipynb   zara.ipynb
H&M_baby_boy.ipynb  H&M_boys.csv	 H&M_girls.csv
H&M_baby_girl.csv   H&M_boys.ipynb	 H&M_girls.ipynb


In [15]:
import pandas as pd
df=pd.read_csv("H&M_boys.csv", encoding='utf-8',index_col=False)
print(df.count())
df.head()

name           4177
brand          4177
gender         4177
category       4177
size           4177
price          4177
composition    4177
color          4177
url            4177
img_url        4177
article_no     4177
blob_url       4177
dtype: int64


,name,brand,gender,category,size,price,composition,color,url,img_url,article_no,blob_url
0,Badshorts med tryck,H&M,boy,badbyxor,134-170,"149,00 kr.",Polyester 100%,Grå/Venice,https://www2.hm.com/sv_se/productpage.04976640...,https://lp2.hm.com/hmgoepprod?set=source[/46/e...,497664001,https://storage.googleapis.com/urbankiddo/H%26...
1,Badshorts med tryck,H&M,boy,badbyxor,134-170,"149,00 kr.",Polyester 100%,Blå/Palmer,https://www2.hm.com/sv_se/productpage.04976640...,https://lp2.hm.com/hmgoepprod?set=source[/2c/1...,497664004,https://storage.googleapis.com/urbankiddo/H%26...
2,Badshorts med tryck,H&M,boy,badbyxor,134-170,"49,90 kr.",Polyester 100%,Svart/Gråmönstrad,https://www2.hm.com/sv_se/productpage.04976640...,https://lp2.hm.com/hmgoepprod?set=source[/27/c...,497664006,https://storage.googleapis.com/urbankiddo/H%26...
3,Badshorts med tryck,H&M,boy,badbyxor,134-170,"69,90 kr.",Polyester 100%,Blå/Hajar,https://www2.hm.com/sv_se/productpage.04976640...,https://lp2.hm.com/hmgoepprod?set=source[/f1/9...,497664007,https://storage.googleapis.com/urbankiddo/H%26...
4,Badshorts med tryck,H&M,boy,badbyxor,134-170,"79,90 kr.",Foder: Polyester 100%,Svart/Stjärnor,https://www2.hm.com/sv_se/productpage.04976640...,https://lp2.hm.com/hmgoepprod?set=source[/c5/0...,497664010,https://storage.googleapis.com/urbankiddo/H%26...


In [16]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json('auth.json')
bucket = storage_client.get_bucket('urbankiddo')
blob = bucket.blob('H&M/H&M_boys.csv')
blob.upload_from_filename('H&M_boys.csv')
blob.public_url

'https://storage.googleapis.com/urbankiddo/H%26M/H%26M_boys.csv'